# --- Prueba de Multiplicador de Lagrange (LM) de Engle para Heterocedasticidad ARCH 🧪 ---

Antes de proceder a aplicar el metodo de  difenreciación (el cual usaremos para tratar ➡ la no estacionariedad de la media (tendencia)) es necesario saber de manera preventiva si la varianza no constante y por tanto existe heterocedasticidad en nuestras series. Para ello he desarrollado este notebook

In [2]:
import pandas as pd
import numpy as np
from statsmodels.stats.diagnostic import het_arch
from statsmodels.tsa.stattools import adfuller

df = pd.read_csv("dataset/pinar_del_rio_temperaturas_LIMPIO_FINAL.csv")

# Convertir la columna 'Fecha' a formato datetime y establecerla como índice
df['Fecha'] = pd.to_datetime(df['Fecha'])
df = df.set_index('Fecha')

print("✅ Datos cargados y preparados exitosamente.")
print(f"Número de observaciones: {len(df)}")

✅ Datos cargados y preparados exitosamente.
Número de observaciones: 671


In [3]:
# --- 2. FUNCIÓN DE PRUEBA DE HETEROCEDASTICIDAD (Test ARCH) ---

def run_arch_test_original(series, name, lags=12):
    """
    Ejecuta el Test ARCH de Engle (LM Test) para heterocedasticidad en la serie original.
    H0: Homocedasticidad (Varianza Constante).
    H1: Heterocedasticidad (Varianza No Constante - Efectos ARCH).
    """
    print(f"\n=======================================================")
    print(f"| PRUEBA ARCH para {name} (Serie ORIGINAL)")
    print(f"=======================================================")
    
    # El test devuelve el estadístico LM, el p-valor, el estadístico F y su p-valor
    # Usamos 12 lags (meses) para buscar autocorrelación en la varianza
    lm_test = het_arch(series, nlags=lags)
    p_value = lm_test[1]
    
    print(f"Estadístico LM (Q): {lm_test[0]:.4f}")
    print(f"p-valor (Test de Engle): {p_value:.4f}")
    
    if p_value < 0.05:
        print("\n➡️ Conclusión: El p-valor (p < 0.05) es significativo.")
        print("   Se **RECHAZA** la H0. **Existe evidencia preliminar de Heterocedasticidad**.")
        print("   La varianza de la serie original es sensible a la magnitud de los shocks pasados.")
    else:
        print("\n➡️ Conclusión: El p-valor (p >= 0.05) no es significativo.")
        print("   **NO se rechaza** la H0. **La varianza es constante** (Homocedasticidad) según este test.")

# --- 3. APLICAR PRUEBAS A LAS SERIES ORIGINALES ---

# 3.1. Prueba ARCH a la Temperatura Máxima (Serie Original)
run_arch_test_original(df['Temp_Maxima_C'].dropna(), 'Temp_Maxima_C')

# 3.2. Prueba ARCH a la Temperatura Mínima (Serie Original)
run_arch_test_original(df['Temp_Minima_C'].dropna(), 'Temp_Minima_C')


| PRUEBA ARCH para Temp_Maxima_C (Serie ORIGINAL)
Estadístico LM (Q): 508.2016
p-valor (Test de Engle): 0.0000

➡️ Conclusión: El p-valor (p < 0.05) es significativo.
   Se **RECHAZA** la H0. **Existe evidencia preliminar de Heterocedasticidad**.
   La varianza de la serie original es sensible a la magnitud de los shocks pasados.

| PRUEBA ARCH para Temp_Minima_C (Serie ORIGINAL)
Estadístico LM (Q): 567.4923
p-valor (Test de Engle): 0.0000

➡️ Conclusión: El p-valor (p < 0.05) es significativo.
   Se **RECHAZA** la H0. **Existe evidencia preliminar de Heterocedasticidad**.
   La varianza de la serie original es sensible a la magnitud de los shocks pasados.


# 📊 Resumen de la Prueba ARCH (Heterocedasticidad)
### Análisis en Series de Temperatura Originales

<table style="width:100%; border-collapse: collapse; text-align: center; font-family: Arial, sans-serif;">
    <thead style="background-color: #f2f2f2;">
        <tr>
            <th style="padding: 12px; border: 1px solid #ddd; font-size: 1.1em;">Serie Analizada</th>
            <th style="padding: 12px; border: 1px solid #ddd; font-size: 1.1em;">Estadístico LM (Q)</th>
            <th style="padding: 12px; border: 1px solid #ddd; font-size: 1.1em;">p-valor (Test de Engle)</th>
            <th style="padding: 12px; border: 1px solid #ddd; font-size: 1.1em;">Conclusión (H0: Homocedasticidad)</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td style="padding: 12px; border: 1px solid #ddd; background-color: #FFFACD; font-weight: bold;">Temp Máxima (°C)</td>
            <td style="padding: 12px; border: 1px solid #ddd;">508.2016</td>
            <td style="padding: 12px; border: 1px solid #ddd; color: red; font-weight: bold;">0.0000</td>
            <td style="padding: 12px; border: 1px solid #ddd; background-color: #FFD7D7; font-weight: bold;">RECHAZADA</td>
        </tr>
        <tr>
            <td style="padding: 12px; border: 1px solid #ddd; background-color: #E0FFFF; font-weight: bold;">Temp Mínima (°C)</td>
            <td style="padding: 12px; border: 1px solid #ddd;">567.4923</td>
            <td style="padding: 12px; border: 1px solid #ddd; color: red; font-weight: bold;">0.0000</td>
            <td style="padding: 12px; border: 1px solid #ddd; background-color: #FFD7D7; font-weight: bold;">RECHAZADA</td>
        </tr>
    </tbody>
</table>

### Implicaciones Prácticas

1.  **Justificación de Transformación:** El resultado de $p=0.0000$ para ambas series es concluyente. Se **rechaza la hipótesis nula de varianza constante**.
2.  **Problema:** Ambas series de temperatura presentan una fuerte **Heterocedasticidad**. Esto significa que la volatilidad (la magnitud de las fluctuaciones de la temperatura) no es constante a lo largo del tiempo.
3.  **Acción Inmediata (Modelado):** La fuerte evidencia de heterocedasticidad **justifica la aplicación de la Transformación Logarítmica ($\ln(Y_t)$)** antes de la diferenciación. El logaritmo es la herramienta estándar para estabilizar la varianza.
4.  **Acción Futura (Avanzada):** Si la heterocedasticidad persiste en los residuos del modelo SARIMA (después de la diferenciación logarítmica), se deberá considerar un modelo **SARIMA-GARCH** para modelar la volatilidad restante.

---

## ➡️ Próximo Paso Lógico

Ahora que hemos confirmado:
1.  **No Estacionariedad en Media** (gráficos ACF/PACF y ADF para Máx).
2.  **No Estacionariedad en Varianza** (Test ARCH).
